In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import ast
import numpy as np
import platform

In [6]:
def make_price_dataframe(code, timeframe, starttime, endtime):
    url = f"https://api.finance.naver.com/siseJson.naver?symbol={code}&requestType=1&startTime={starttime}&endTime={endtime}&timeframe={timeframe}"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    price_data_str = str(soup).replace("\t","").replace("\n","").replace(" ","")
    price_data = ast.literal_eval(price_data_str)
    price_data = price_data[1:]
    date_list = []
    market_list = []
    closing_list = []
    high_list = []
    low_list = []
    volume_list = []
    for item in price_data:
        date_list.append(item[0])
        market_list.append(item[1])
        high_list.append(item[2])
        low_list.append(item[3])
        closing_list.append(item[4])
        volume_list.append(item[5])
    price_df = pd.DataFrame({('open',code):market_list, ('high',code):high_list, 
                            ('low',code):low_list, ('close',code):closing_list,  
                            ('volume',code):volume_list,},
                            index=date_list)
    return price_df


def make_code(x):
    x = str(x)
    return "0"*(6-len(x)) + x


In [7]:
path = pd.read_csv("korea_corporation.csv")
code_data = path[["단축코드","한글 종목약명"]][path["주식종류"] == "보통주"]
code_data["단축코드"] = code_data["단축코드"].apply(make_code)


------------
# 날짜 수정(필수!!)

In [1]:
start = '20211102'
end = '20211121'

------------

# 데이터 크롤링(시간 오래걸림)

In [10]:
for num, code in enumerate(code_data["단축코드"]):
    try:
        print(num, code)
        time.sleep(1)
        try:
            price_df = make_price_dataframe(code,"day", start, end)
        except requests.exceptions.Timeout:
            time.sleep(60)
            price_df = make_price_dataframe(code,"day", start, end)
        if num == 0:
            total_price = price_df
        else:
            total_price = pd.merge(total_price,price_df,how='outer',right_index=True,left_index=True)
    except ValueError:
        print("valueError")
        continue
    except KeyError:
        print("keyError")
        continue

0 098120
1 131100
2 009520
3 095570
4 006840
5 282330
6 027410
7 138930
8 001460
9 001040
10 011150
11 058820
12 000590
13 012030
14 016610
15 005830
16 000990
17 139130
18 001530
19 001880
20 000210
21 375500
22 155660
23 069730
24 017940
25 365550
26 050120
27 383220
28 007700
29 006360
30 078930
31 012630
32 294870
33 003560
34 175330
35 234080
36 001060
37 096760
38 105560
39 024120
40 009070
41 044180
42 016380
43 001390
44 033180
45 001940
46 025000
47 092230
48 000040
49 093050
50 034220
51 003550
52 051900
53 032640
54 011070
55 066570
56 051910
57 079550
58 010120
59 000680
60 006260
61 001120
62 023150
63 035420
64 181710
65 338100
66 034310
67 008260
68 004250
69 010060
70 010950
71 950110
72 101060
73 005090
74 001380
75 001770
76 002360
77 009160
78 123700
79 025530
80 011790
81 018670
82 001740
83 210980
84 395400
85 034730
86 361610
87 096770
88 001510
89 285130
90 017670
91 064960
92 100840
93 003570
94 036530
95 005610
96 011810
97 077970
98 071970
99 084870
100 002710

In [11]:
index = []
new_code = []
for col in total_price.columns:
    index.append(col[0])
    new_code.append('A' + col[1])

total_price.columns = [index,new_code]

# 파일 처음 만들 때

In [ ]:
# 주석 지워 사용
# total_price.to_csv('stock_price.csv')

# 원래 파일에 붙일 때

In [13]:
# 주석 지워 사용
# old_price = pd.read_csv('stock_price.csv', header=[0,1], index_col=0)
# pd.concat([old_price, total_price]).to_csv('stock_price.csv')

# 저장된 데이터 파일 읽어올 때 기본 설정

In [15]:
# price_df = pd.read_csv('stock_price.csv', header=[0,1], index_col=0)